In [30]:
# import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import sklearn
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
#
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
#
import os
import numpy as np
import scipy.stats as stats
from statsmodels.formula.api import ols
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
#
from sklearn.preprocessing import LabelEncoder
make_lb = LabelEncoder()
#
from sklearn.preprocessing import LabelBinarizer
lbb = LabelBinarizer()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV

In [26]:
!pip install jupyterthemes

In [28]:
!jt -t oceans16
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [29]:
!jt -t monokai

In [31]:
with open('df_trimmed.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df_final = pickle.load(f)
df_final.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,default payment next month,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,PAY_0N1,PAY_0N2
0,2873.0,350000.0,1.0,1.0,2.0,37.0,0,0.0,0.001046,0.001046,0.0,0.000000,0.0,0,0
1,3598.0,50000.0,2.0,2.0,1.0,37.0,2,0.0,0.542003,0.026267,0.0,0.000000,0.0,0,0
2,27623.0,50000.0,2.0,1.0,2.0,23.0,0,0.0,0.078257,0.077063,2.0,0.011310,0.0,1,1
3,6874.0,20000.0,1.0,3.0,1.0,56.0,0,0.0,0.639975,0.041567,0.0,0.000000,1.0,0,0
4,6444.0,110000.0,2.0,2.0,2.0,32.0,0,0.0,0.983186,0.041364,1.0,0.012245,0.0,0,0


In [32]:
df_final.drop(columns=['ID'], axis=1, inplace=True)

### General Layout

`rfecv = RFECV(estimator=MODEL, step=1, cv=,
              scoring='accuracy')`

In [33]:
df_final.rename(columns={'default payment next month':'default'}, inplace=True)
df_final.columns=df_final.columns.str.lower()
df_final.head()

,limit_bal,sex,education,marriage,age,pay_0,default,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,pay_0n1,pay_0n2
0,350000.0,1.0,1.0,2.0,37.0,0,0.0,0.001046,0.001046,0.0,0.000000,0.0,0,0
1,50000.0,2.0,2.0,1.0,37.0,2,0.0,0.542003,0.026267,0.0,0.000000,0.0,0,0
2,50000.0,2.0,1.0,2.0,23.0,0,0.0,0.078257,0.077063,2.0,0.011310,0.0,1,1
3,20000.0,1.0,3.0,1.0,56.0,0,0.0,0.639975,0.041567,0.0,0.000000,1.0,0,0
4,110000.0,2.0,2.0,2.0,32.0,0,0.0,0.983186,0.041364,1.0,0.012245,0.0,0,0


### I have to run my three base models with this data in order to see how it performs once I start to use a feature elimination module.


---

### Baseline Logistic Regression Model

In [34]:
# Preparing data for the model
df_fin_mms = df_final
df_fin_mms = minmax.fit_transform(df_fin_mms)

In [35]:
df_fin_mms = pd.DataFrame(df_fin_mms, columns=df_final.columns)
df_fin_mms.head()

,limit_bal,sex,education,marriage,age,pay_0,default,avg_utilization,avg_payment_impact,debt_streak,debt_avg_delta,payment_improvements,pay_0n1,pay_0n2
0,0.343434,0.0,0.166667,0.666667,0.275862,0.00,0.0,0.036158,0.000431,0.0,0.000000,0.0,0.0,0.0
1,0.040404,1.0,0.333333,0.333333,0.275862,0.25,0.0,0.133375,0.010819,0.0,0.000000,0.0,0.0,0.0
2,0.040404,1.0,0.166667,0.666667,0.034483,0.00,0.0,0.050034,0.031743,0.4,0.004299,0.0,1.0,1.0
3,0.010101,0.0,0.500000,0.333333,0.603448,0.00,0.0,0.150981,0.017122,0.0,0.000000,0.2,0.0,0.0
4,0.101010,1.0,0.333333,0.666667,0.189655,0.00,0.0,0.212660,0.017038,0.2,0.004655,0.0,0.0,0.0


In [36]:
y = df_fin_mms['default']
X = df_fin_mms.drop(columns=['default'], axis=1)

In [37]:
logreg_base = LogisticRegressionCV(random_state=42, cv=5, scoring = 'accuracy', solver='saga')
base_lr = logreg_base.fit(X, y)

In [38]:
cross_validate(logreg_base, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([3.27245998, 3.55791211, 3.87258101, 3.84000468, 3.85964918]),
 'score_time': array([0.02216125, 0.01417589, 0.01464581, 0.0167563 , 0.01533484]),
 'test_f1': array([0.46029777, 0.45083014, 0.47195122, 0.42610365, 0.48406863]),
 'test_recall': array([0.34934087, 0.33239171, 0.36440678, 0.31385485, 0.37229029]),
 'test_precision': array([0.67454545, 0.70039683, 0.66955017, 0.66334661, 0.69176883]),
 'test_accuracy': array([0.81878775, 0.82083333, 0.81958333, 0.81308606, 0.82454678])}

In [39]:
# Probably a better way of doing this
test_f1 = np.mean([0.46029777, 0.45083014, 0.47195122, 0.42637644, 0.48406863])
test_recall = np.mean([0.34934087, 0.33239171, 0.36440678, 0.31385485, 0.37229029])
test_precision = np.mean([0.67454545, 0.70039683, 0.66955017, 0.66467066, 0.69176883])
test_accuracy = np.mean([0.81878775, 0.82083333, 0.81958333, 0.81329444, 0.82454678])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.45870484000000006
test_recall: 0.3464569
test_precision: 0.680186388
test_accuracy: 0.819409126


---

### Baseline KNN Model

In [40]:
knn = KNeighborsClassifier()

In [41]:
cross_validate(knn, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([0.11845398, 0.08772993, 0.08167505, 0.16339302, 0.0830369 ]),
 'score_time': array([1.6751101 , 1.72525215, 1.74677682, 1.8876617 , 1.78995395]),
 'test_f1': array([0.41520468, 0.42747188, 0.41743119, 0.41852286, 0.43367935]),
 'test_recall': array([0.33427495, 0.33992467, 0.34274953, 0.33647502, 0.35438266]),
 'test_precision': array([0.54783951, 0.57575758, 0.53372434, 0.55348837, 0.55869242]),
 'test_accuracy': array([0.79171006, 0.79854167, 0.78833333, 0.79329027, 0.79537404])}

In [42]:
# Probably a better way of doing this
test_f1 = np.mean([0.41520468, 0.42747188, 0.41743119, 0.41852286, 0.43367935])
test_recall = np.mean([0.33427495, 0.33992467, 0.34274953, 0.33647502, 0.35438266])
test_precision = np.mean([0.54783951, 0.57575758, 0.53372434, 0.55348837, 0.55869242])
test_accuracy = np.mean([0.79171006, 0.79854167, 0.78833333, 0.79329027, 0.79537404])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.422461992
test_recall: 0.341561366
test_precision: 0.5539004439999999
test_accuracy: 0.793449874


---

### DecisionTreeClassifier Base Model

In [43]:
dt_base = DecisionTreeClassifier(random_state=42)

In [44]:
cross_validate(dt_base, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([0.1593101 , 0.13966894, 0.14208317, 0.14480495, 0.13841987]),
 'score_time': array([0.01849008, 0.02695298, 0.01778984, 0.018466  , 0.01734805]),
 'test_f1': array([0.37683824, 0.38942976, 0.406165  , 0.38906606, 0.43054935]),
 'test_recall': array([0.38606403, 0.39548023, 0.42184557, 0.40245052, 0.45428841]),
 'test_precision': array([0.36804309, 0.38356164, 0.39160839, 0.37654321, 0.40916808]),
 'test_accuracy': array([0.71755884, 0.725625  , 0.72708333, 0.72056678, 0.73431965])}

In [45]:
test_f1 = np.mean([0.37683824, 0.38942976, 0.406165  , 0.38906606, 0.43054935])
test_recall = np.mean([0.38606403, 0.39548023, 0.42184557, 0.40245052, 0.45428841])
test_precision = np.mean([0.36804309, 0.38356164, 0.39160839, 0.37654321, 0.40916808])
test_accuracy = np.mean([0.71755884, 0.725625  , 0.72708333, 0.72056678, 0.73431965])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)

test_f1: 0.39840968199999993
test_recall: 0.4120257519999999
test_precision: 0.38578488199999994
test_accuracy: 0.72503072


---

### Time to check out RFECV


In [46]:
# KNN doesn't support this
rfecv_lg = RFECV(estimator=logreg_base, step=1, min_features_to_select=10, cv=5,
              scoring='accuracy')
rfecv_dt = RFECV(estimator=dt_base, step=1, min_features_to_select=10, cv=5,
              scoring='accuracy')

In [47]:
rfecv_lg.fit(X,y)
rfecv_dt.fit(X,y)

RFECV(cv=5,
      estimator=DecisionTreeClassifier(class_weight=None, criterion='gini',
                                       max_depth=None, max_features=None,
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort=False, random_state=42,
                                       splitter='best'),
      min_features_to_select=10, n_jobs=None, scoring='accuracy', step=1,
      verbose=0)

In [48]:
lg_feats = pd.DataFrame(rfecv_lg.ranking_)
dt_feats = pd.DataFrame(rfecv_dt.ranking_)

In [49]:
# Generating the feature selection for LogReg
lg_feats = lg_feats.T
lg_feats.rename(columns={0:X.columns[0], 1:X.columns[1], 2:X.columns[2], 3:X.columns[3], 4:X.columns[4],
                         5:X.columns[5], 6:X.columns[6], 7:X.columns[7], 8:X.columns[8], 9:X.columns[9],
                         10:X.columns[10], 11:X.columns[11], 12:X.columns[12]}, inplace=True)

In [50]:
lg_feats = lg_feats.T
lg_feats.rename(columns={0:'Feature Ranks'}, inplace=True)
lg_feats

,Feature Ranks
limit_bal,1
sex,1
education,1
marriage,1
age,1
pay_0,1
avg_utilization,1
avg_payment_impact,1
debt_streak,1
debt_avg_delta,2


In [51]:
# Generating the feature selection for DecTree
dt_feats = dt_feats.T
dt_feats.rename(columns={0:X.columns[0], 1:X.columns[1], 2:X.columns[2], 3:X.columns[3], 4:X.columns[4],
                         5:X.columns[5], 6:X.columns[6], 7:X.columns[7], 8:X.columns[8], 9:X.columns[9],
                         10:X.columns[10], 11:X.columns[11], 12:X.columns[12]}, inplace=True)

In [52]:
dt_feats = dt_feats.T
dt_feats.rename(columns={0:'Feature Ranks'}, inplace=True)
dt_feats

,Feature Ranks
limit_bal,1
sex,1
education,1
marriage,1
age,1
pay_0,1
avg_utilization,1
avg_payment_impact,1
debt_streak,1
debt_avg_delta,1


### 1 means important, with larger numbers indicating less importance


---

## Model Refinement | Hyperparameter Tuning

---

### LogReg Refinement

#### Tunable Params
* Cs=10
* fit_intercept=True
* cv='warn'
* dual=False
* penalty='l2'
* scoring=None
* solver='lbfgs'
* tol=0.0001
* max_iter=100
* class_weight=None
* n_jobs=None
* verbose=0
* refit=True
* intercept_scaling=1.0
* multi_class='warn'
* random_state=None
* l1_ratios=None

In [53]:
param_grid = {
    'Cs':[4, 8, 10, 12],
    'cv':[3, 5, 8],
    'penalty':['l1', 'l2'],
    'solver':['liblinear', 'saga'],
    'class_weight': ['balanced'],
    'random_state':[42]
}

In [60]:
gridsearch = GridSearchCV(estimator=logreg_base, param_grid=param_grid)
gridsearch.fit(X.drop(columns=['debt_avg_delta'], axis=1),y)

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LogisticRegressionCV(Cs=10, class_weight=None, cv=5,
                                            dual=False, fit_intercept=True,
                                            intercept_scaling=1.0,
                                            l1_ratios=None, max_iter=100,
                                            multi_class='warn', n_jobs=None,
                                            penalty='l2', random_state=42,
                                            refit=True, scoring='accuracy',
                                            solver='saga', tol=0.0001,
                                            verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'Cs': [4, 8, 10, 12], 'class_weight': ['balanced'],
                         'cv': [3, 5, 8], 'penalty': ['l1', 'l2'],
                         'random_state': [42],
                         'solver': ['liblinear', 'saga']},
       

In [61]:
gridsearch.best_params_

{'Cs': 4,
 'class_weight': 'balanced',
 'cv': 3,
 'penalty': 'l1',
 'random_state': 42,
 'solver': 'liblinear'}

## Best LogReg Model
`l1` penalty seemed to cause some problems with scoring, so I am going to leave this as `l2`

In [62]:
lg_refined = LogisticRegressionCV(Cs=4, class_weight='balanced', cv=3, penalty='l2',
                                 random_state=42, solver='liblinear')
lg_refined.fit(X,y)

LogisticRegressionCV(Cs=4, class_weight='balanced', cv=3, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='warn', n_jobs=None,
                     penalty='l2', random_state=42, refit=True, scoring=None,
                     solver='liblinear', tol=0.0001, verbose=0)

In [63]:
cross_validate(lg_refined, X, y, scoring=['f1', 'recall', 'precision', 'accuracy', 'roc_auc'], cv=5)

{'fit_time': array([0.55793118, 0.56308007, 0.60768294, 0.53129888, 0.52650809]),
 'score_time': array([0.01775193, 0.02073693, 0.02929926, 0.01945424, 0.01862502]),
 'test_f1': array([0.50927924, 0.51559792, 0.51904563, 0.50568679, 0.5318872 ]),
 'test_recall': array([0.55555556, 0.56026365, 0.58380414, 0.54476909, 0.57775683]),
 'test_precision': array([0.47011952, 0.47752809, 0.46721929, 0.47183673, 0.49276527]),
 'test_accuracy': array([0.76317434, 0.76708333, 0.760625  , 0.76453428, 0.77516149]),
 'test_roc_auc': array([0.74912751, 0.7548469 , 0.75377152, 0.74442804, 0.76244422])}

In [64]:
test_f1 = np.mean([0.50927924, 0.51559792, 0.51904563, 0.50568679, 0.5318872 ])
test_recall = np.mean([0.55555556, 0.56026365, 0.58380414, 0.54476909, 0.57775683])
test_precision = np.mean([0.47011952, 0.47752809, 0.46721929, 0.47183673, 0.49276527])
test_accuracy = np.mean([0.76317434, 0.76708333, 0.760625  , 0.76453428, 0.77516149])
roc_auc = np.mean([0.74912751, 0.7548469 , 0.75377152, 0.74442804, 0.76244422])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)
print('roc_auc:', roc_auc)

test_f1: 0.516299356
test_recall: 0.5644298539999999
test_precision: 0.47589378
test_accuracy: 0.766115688
roc_auc: 0.752923638


While this model did suffer in certain evaluation metrics compared to the base, I saw a considerable increase in the F1 score. 

---

### KNN Refinement

#### Tunable Params

* algorithm='auto'
* leaf_size=30
* metric='minkowski'
* metric_params=None
* n_jobs=None
* n_neighbors=5
* p=2
* weights='uniform'

In [97]:
param_grid = {
    'algorithm':['auto','ball_tree', 'kd_tree', 'brute'],
    'weights':['uniform', 'distance'],
    'p':[1,2],
    'leaf_size':[20, 30, 40],
    'n_neighbors':[2, 5, 10]
}

In [98]:
# Going to use all features
gridsearch = GridSearchCV(estimator=knn, param_grid=param_grid, cv=3)
gridsearch.fit(X,y)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [20, 30, 40], 'n_neighbors': [2, 5, 10],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [99]:
gridsearch.best_params_

{'algorithm': 'auto',
 'leaf_size': 20,
 'n_neighbors': 10,
 'p': 2,
 'weights': 'uniform'}

In [100]:
knn_ref = KNeighborsClassifier(n_neighbors = 10)
cross_validate(knn_ref, X, y, scoring=['f1', 'recall', 'precision', 'accuracy'], cv=5)

{'fit_time': array([0.11325788, 0.07064199, 0.06844592, 0.12845707, 0.07187819]),
 'score_time': array([1.78325415, 1.69043803, 1.69204521, 1.7538228 , 1.67548966]),
 'test_f1': array([0.38925295, 0.42077581, 0.39164491, 0.38258575, 0.41428571]),
 'test_recall': array([0.27966102, 0.30131827, 0.28248588, 0.27332705, 0.30065975]),
 'test_precision': array([0.64008621, 0.69716776, 0.63829787, 0.63736264, 0.66597077]),
 'test_accuracy': array([0.80587378, 0.81645833, 0.80583333, 0.80495937, 0.81204418])}

In [101]:
test_f1 = np.mean([0.38925295, 0.42077581, 0.39164491, 0.38258575, 0.41428571])
test_recall = np.mean([0.27966102, 0.30131827, 0.28248588, 0.27332705, 0.30065975])
print('test_f1:', test_f1)
print('test_recall:', test_recall)

test_f1: 0.39970902599999997
test_recall: 0.28749039400000004


---

### DecisionTree Refinement

#### Tunable Params
* criterion
* splitter
* max_depth
* min_samples_split
* min_samples_leaf
* class_weight

In [93]:
param_grid = {
    'criterion':['gini', 'entropy'],
    'max_depth':[3, 5, 7],
    'class_weight':[None, 'balanced'],
    'random_state':[42],
    'min_samples_leaf':[1,2,5],
    'min_samples_split':[2,3,4,6]
}

In [94]:
gridsearch = GridSearchCV(estimator=dt_base, param_grid=param_grid, cv=3)
gridsearch.fit(X.drop(columns=['pay_0n1', 'pay_0n2'], axis=1),y)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'class_weight': [None, 'balanced'],
                         'criterion': ['gini', 'entropy'],
                  

In [95]:
gridsearch.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'random_state': 42}

There was a change in the `max_depth` parameter. Will be setting `max_depth` to 3.

In [85]:
dt_refined = DecisionTreeClassifier(class_weight = None,
                                    criterion = 'gini', max_depth = 3,
                                    random_state = 42)
# Going to drop the columns that the RFECV selector scored 2 or 3
dt_refined.fit(X.drop(columns=['pay_0n1', 'pay_0n2'], axis=1),y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [86]:
cross_validate(dt_refined, X.drop(columns=['pay_0n1', 'pay_0n2'], axis=1), y, scoring=['f1', 'recall', 'precision', 'accuracy', 'roc_auc'], cv=5)

{'fit_time': array([0.05379486, 0.03170371, 0.03180695, 0.02682185, 0.02729106]),
 'score_time': array([0.0265429 , 0.01683712, 0.01570797, 0.01547122, 0.015692  ]),
 'test_f1': array([0.44684914, 0.4396608 , 0.45402663, 0.41655716, 0.46835443]),
 'test_recall': array([0.33050847, 0.3173258 , 0.33709981, 0.29877474, 0.34872762]),
 'test_precision': array([0.68958743, 0.71549894, 0.69514563, 0.68763557, 0.71290944]),
 'test_accuracy': array([0.81899604, 0.82104167, 0.820625  , 0.81496145, 0.82496353]),
 'test_roc_auc': array([0.72330827, 0.7267817 , 0.73206741, 0.7171861 , 0.73633945])}

In [87]:
test_f1 = np.mean([0.44684914, 0.4396608 , 0.45402663, 0.41655716, 0.46835443])
test_recall = np.mean([0.33050847, 0.3173258 , 0.33709981, 0.29877474, 0.34872762])
test_precision = np.mean([0.68958743, 0.71549894, 0.69514563, 0.68763557, 0.71290944])
test_accuracy = np.mean([0.81899604, 0.82104167, 0.820625  , 0.81496145, 0.82496353])
roc_auc = np.mean([0.72330827, 0.7267817 , 0.73206741, 0.7171861 , 0.73633945])
print('test_f1:', test_f1)
print('test_recall:', test_recall)
print('test_precision:', test_precision)
print('test_accuracy:', test_accuracy)
print('roc_auc:', roc_auc)

test_f1: 0.445089632
test_recall: 0.326487288
test_precision: 0.700155402
test_accuracy: 0.8201175380000001
roc_auc: 0.7271365860000001


Can turn this into a graph for the final project

## LogReg Base
test_f1: 0.45870484000000006  
test_recall: 0.3464569     
  
## KNN Base
test_f1: 0.422461992  
test_recall: 0.341561366  

## DTC Base
test_f1: 0.39840968199999993  
test_recall: 0.4120257519999999  

---

## LogReg Refined **
**test_f1: 0.516299356**  
**test_recall: 0.5644298539999999**

## KNN Refined
test_f1: 0.39970902599999997  
test_recall: 0.28749039400000004  

## DTC Refined
test_f1: 0.445089632  
test_recall: 0.326487288  

## Conclusion
The best model was the LogReg Refined model, scoring the highest F1 and Recall. These two metrics were chosen to be the deciding factors, with more weight being given to F1, because the cost of not forseeing if someone will default on their credit cards due to a False Negative is higher than mistakenly flagging someone that they might be on track to default on their credit cards. This may actually benefit some customers who are tetering on that fence of poor credit history, and thus could benefit from resources to help them better their credit or also benefit the credit company by not allowing the individual to open a line.
